In [1]:
from PIL import Image
import os

# === Config ===
image_path = "input_data/X_test/"  # Đường dẫn đến ảnh gốc
ROWS = 3
COLS = 5

def split_image(image_path, rows, cols):
    """Đọc ảnh và cắt thành rows x cols, trả về danh sách các mảnh."""
    img = Image.open(image_path)
    width, height = img.size

    piece_width = width // cols
    piece_height = height // rows

    pieces = []
    for r in range(rows):
        for c in range(cols):
            left = c * piece_width
            upper = r * piece_height
            right = left + piece_width
            lower = upper + piece_height
            piece = img.crop((left, upper, right, lower))
            pieces.append(piece)

    return pieces

def load_all_pieces(image_dir, rows, cols):
    """
    Duyệt toàn bộ ảnh trong thư mục và trả về mảng 3 chiều:
    all_pieces[image_index][row][col] = PIL.Image
    """
    all_pieces = []
    image_files = sorted(
        [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    )

    print(f"Found {len(image_files)} images in {image_dir}")

    for filename in image_files:
        image_path = os.path.join(image_dir, filename)
        print(f"- Processing {filename}...")
        pieces = split_image(image_path, rows, cols)
        all_pieces.append(pieces)

    print(f"\nTotal images processed: {len(all_pieces)}")
    return all_pieces

all_pieces = load_all_pieces(image_path, ROWS, COLS)
# if DEBUG:
#     all_pieces = all_pieces[:20]  # Giới hạn để debug nhanh

# Ví dụ: xem thông tin
print(f"Total images: {len(all_pieces)}")  # số lượng ảnh
if all_pieces:
    print(f"Rows per image: {len(all_pieces[0])}")


Found 100 images in input_data/X_test/
- Processing Alfred_Sisley_115_shuffled.jpg...
- Processing Alfred_Sisley_188_shuffled.jpg...
- Processing Alfred_Sisley_205_shuffled.jpg...
- Processing Alfred_Sisley_232_shuffled.jpg...
- Processing Alfred_Sisley_6_shuffled.jpg...
- Processing Amedeo_Modigliani_112_shuffled.jpg...
- Processing Amedeo_Modigliani_143_shuffled.jpg...
- Processing Amedeo_Modigliani_2_shuffled.jpg...
- Processing Amedeo_Modigliani_65_shuffled.jpg...
- Processing Amedeo_Modigliani_73_shuffled.jpg...
- Processing Andrei_Rublev_32_shuffled.jpg...
- Processing Andrei_Rublev_50_shuffled.jpg...
- Processing Andrei_Rublev_89_shuffled.jpg...
- Processing Andy_Warhol_11_shuffled.jpg...
- Processing Andy_Warhol_161_shuffled.jpg...
- Processing Andy_Warhol_171_shuffled.jpg...
- Processing Andy_Warhol_70_shuffled.jpg...
- Processing Andy_Warhol_88_shuffled.jpg...
- Processing Camille_Pissarro_3_shuffled.jpg...
- Processing Caravaggio_19_shuffled.jpg...
- Processing Caravaggio_22

In [2]:
import numpy as np
from PIL import Image
import os
import csv
import random
from tqdm import tqdm
import pandas as pd
# =========================
# COST: MSE theo viền kề
# =========================
def _to_np_rgb(pil_img):
    return np.asarray(pil_img.convert("RGB"), dtype=np.float32)

def _border_arrays(pieces):
    arrs = [_to_np_rgb(p) for p in pieces]
    tops    = [a[0, :, :]  for a in arrs]  # (W,3)
    bottoms = [a[-1, :, :] for a in arrs]  # (W,3)
    lefts   = [a[:, 0, :]  for a in arrs]  # (H,3)
    rights  = [a[:, -1, :] for a in arrs]  # (H,3)
    return tops, bottoms, lefts, rights

def _mse_rgb_edge(edge1, edge2):
    L = min(edge1.shape[0], edge2.shape[0])
    if L <= 0:
        return 0.0
    diff = edge1[:L].astype(np.float32) - edge2[:L].astype(np.float32)
    return float(np.mean(diff * diff))

def compute_cost_matrix_mse(pieces):
    """
    - H[i,j]: cost đặt j bên phải i  -> so sánh right(i) vs left(j)
    - V[i,j]: cost đặt j bên dưới i -> so sánh bottom(i) vs top(j)
    """
    n = len(pieces)
    H = np.zeros((n, n), dtype=np.float64)
    V = np.zeros((n, n), dtype=np.float64)

    tops, bottoms, lefts, rights = _border_arrays(pieces)
    for i in range(n):
        r_i = rights[i]
        b_i = bottoms[i]
        for j in range(n):
            if i == j:
                continue
            H[i, j] = _mse_rgb_edge(r_i, lefts[j])
            V[i, j] = _mse_rgb_edge(b_i, tops[j])
    return H, V
# =========================
# MST-BASED SOLVER
# =========================
class DSU:
    def __init__(self, n):
        self.p = list(range(n))
        self.r = [0]*n
    def find(self, x):
        while self.p[x] != x:
            self.p[x] = self.p[self.p[x]]
            x = self.p[x]
        return x
    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb: return False
        if self.r[ra] < self.r[rb]:
            self.p[ra] = rb
        elif self.r[ra] > self.r[rb]:
            self.p[rb] = ra
        else:
            self.p[rb] = ra
            self.r[ra] += 1
        return True

def invert_perm_c2o_to_o2c(c2o):
    """c2o[c]=o  ->  o2c[o]=c"""
    n = len(c2o)
    o2c = [0]*n
    for c, o in enumerate(c2o):
        o2c[o] = c
    return o2c

def invert_perm_o2c_to_c2o(o2c):
    """o2c[o]=c  ->  c2o[c]=o"""
    n = len(o2c)
    c2o = [0]*n
    for o, c in enumerate(o2c):
        c2o[c] = o
    return c2o

# =========================
# SPECTRAL-BASED SOLVER (2D)
# =========================
import numpy as np

def _median_offdiag(A):
    n = A.shape[0]
    mask = ~np.eye(n, dtype=bool)
    vals = A[mask]
    # phòng trường hợp số cực đoan
    m = np.median(vals) if vals.size else 1.0
    return max(float(m), 1e-6)

def _fiedler_vector(S):
    """Trả về eigenvector nhỏ thứ 2 của Laplacian D-S (đối xứng).
       Giữ vector chuẩn hoá (chuẩn không quan trọng, chỉ thứ tự)."""
    # Đảm bảo đối xứng số học
    S = 0.5 * (S + S.T)
    np.fill_diagonal(S, 0.0)
    d = np.sum(S, axis=1)
    L = np.diag(d) - S
    # eigh cho ma trận đối xứng
    w, U = np.linalg.eigh(L)
    # Bỏ eigenvector hằng (ứng với eigenvalue ~0). Lấy vector thứ 2.
    # (Trong thực tế, eigenvalues có thể 0 lặp do thành phần rời -> xử lý bảo thủ)
    idx = np.argsort(w)
    U = U[:, idx]
    # chọn vector phi-hằng đầu tiên
    # (nếu nhiều zero-eig, lấy cột đầu tiên không gần hằng)
    for k in range(1, U.shape[1]):
        v = U[:, k]
        # kiểm tra “độ biến thiên” để tránh chọn vector gần hằng
        if np.std(v) > 1e-12:
            return v
    return U[:, 1]  # fallback

class SpectralPuzzleSolver:
    """
    Ghép ảnh bằng spectral embedding 2 trục:
      - Trục x (cột): dùng similarity từ H (ngang).
      - Trục y (hàng): dùng similarity từ V (dọc).
    Mapping:
      1) Sắp theo f_x -> chia thành C "cột" (mỗi cột R mảnh).
      2) Trong từng cột, sắp theo f_y -> gán hàng 0..R-1.
    Trả về chromosome c->o (cell->piece) theo row-major.
    """
    def __init__(self, rows, cols, tau_scale=1.0):
        self.rows = rows
        self.cols = cols
        self.tau_scale = tau_scale

    def _to_similarity(self, Dmat):
        """Từ dissimilarity đối xứng -> similarity qua exp(-D/tau)."""
        # đảm bảo đối xứng
        Dsym = 0.5 * (Dmat + Dmat.T)
        np.fill_diagonal(Dsym, np.inf)  # không tự nối
        tau = self.tau_scale * _median_offdiag(Dsym)
        # tránh overflow: exp(-inf/tau)=0 là ok
        S = np.exp(-Dsym / tau)
        np.fill_diagonal(S, 0.0)
        return S

    def run(self, pieces):
        n = len(pieces)
        assert n == self.rows * self.cols, "Số mảnh không khớp rows*cols"

        # 1) Tính H,V
        H, V = compute_cost_matrix_mse(pieces)

        # 2) Similarity cho ngang (Sx) & dọc (Sy)
        #    D_x = H+H^T; D_y = V+V^T (độ không tương thích đôi chiều)
        Dx = H + H.T
        Dy = V + V.T
        Sx = self._to_similarity(Dx)
        Sy = self._to_similarity(Dy)

        # 3) Lấy Fiedler vector cho mỗi trục
        fx = _fiedler_vector(Sx)  # dùng để sắp theo cột
        fy = _fiedler_vector(Sy)  # dùng để sắp theo hàng

        # 4) Sắp theo fx -> chia làm C cột (mỗi cột R mảnh)
        order_x = np.argsort(fx)  # từ trái -> phải
        grid = [[None for _ in range(self.cols)] for __ in range(self.rows)]

        for c in range(self.cols):
            col_pieces = order_x[c*self.rows:(c+1)*self.rows]
            # 5) Trong cột, sắp theo fy -> từ trên xuống dưới
            col_pieces_sorted = sorted(col_pieces, key=lambda p: fy[p])
            for r in range(self.rows):
                grid[r][c] = int(col_pieces_sorted[r])

        # 6) Xuất c->o (cell -> piece) theo row-major
        c2o = []
        for r in range(self.rows):
            for c in range(self.cols):
                c2o.append(grid[r][c])

        # (tuỳ chọn) ước lượng cost để log
        def approx_cost(c2o_arr):
            tot = 0.0
            g = np.array(c2o_arr, dtype=int).reshape(self.rows, self.cols)
            for r in range(self.rows):
                for c in range(self.cols):
                    cur = g[r, c]
                    if c < self.cols - 1:
                        tot += H[cur, g[r, c+1]]
                    if r < self.rows - 1:
                        tot += V[cur, g[r+1, c]]
            return float(tot)

        return c2o, approx_cost(c2o)

    def assemble_image(self, pieces, order):
        # giống hàm assemble_image của bạn/MST
        n = self.rows * self.cols
        if len(order) != n or set(order) != set(range(n)):
            used, out = set(), []
            for g in order:
                if isinstance(g, (int, np.integer)) and 0 <= g < n and g not in used:
                    out.append(int(g)); used.add(int(g))
            out.extend([x for x in range(n) if x not in used])
            order = out[:n]
        grid = np.array(order, dtype=int).reshape(self.rows, self.cols)
        w, h = pieces[0].size
        out = Image.new("RGB", (w*self.cols, h*self.rows))
        for r in range(self.rows):
            for c in range(self.cols):
                out.paste(pieces[grid[r, c]], (c*w, r*h))
        return out



class PuzzleRunnerLocal:
    def __init__(self, solver, pieces_list, image_dir, output_dir, output_img_dir, y_true_csv):
        self.solver = solver
        self.pieces_list = pieces_list
        self.image_dir = image_dir
        self.output_dir = output_dir
        self.output_img_dir = output_img_dir
        self.y_true_csv = y_true_csv
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.output_img_dir, exist_ok=True)

    def run_all(self):
        results = []
        image_files = sorted([f for f in os.listdir(self.image_dir)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        for idx, pieces in enumerate(tqdm(self.pieces_list, desc="Local search for images")):
            best_order, _ = self.solver.run(pieces)  # c->o (cell->piece)
            image_name = image_files[idx]

            # Lưu ảnh lắp theo c->o
            img = self.solver.assemble_image(pieces, best_order)
            img.save(os.path.join(self.output_img_dir, f"{image_name}_solved.png"))

            # Ghi trực tiếp c->o ra CSV (rất tự nhiên: piece_at_r_c)
            results.append([image_name] + best_order[:])

        # Ghi output.csv (c->o)
        output_csv = os.path.join(self.output_dir, "output.csv")
        with open(output_csv, "w", newline="") as f:
            writer = csv.writer(f)
            header = ["image_filename"] + [
                f"piece_at_{r}_{c}" for r in range(self.solver.rows) for c in range(self.solver.cols)
            ]
            writer.writerow(header)
            writer.writerows(results)

        print(f"Saved output to {output_csv}")
        print(f"Solved images saved to {self.output_img_dir}")
        return output_csv

    def evaluate(self, output_csv):
        df_pred = pd.read_csv(output_csv)
        df_true = pd.read_csv(self.y_true_csv)

        correct_count = 0
        ppa_scores = []

        true_map = {row['image_filename']: row.values[1:].astype(int)
                    for _, row in df_true.iterrows()}

        for _, row in df_pred.iterrows():
            fname = row['image_filename']
            if fname not in true_map:
                continue
            pred = row.values[1:].astype(int)  # chúng ta xuất c->o
            gt   = true_map[fname]            # chưa chắc c->o hay o->c

            # 1) GT là c->o: so trực tiếp
            match_direct = (pred == gt).sum()

            # 2) GT là o->c: đảo về c->o rồi so
            if set(gt) == set(range(len(gt))):
                gt_as_c2o = np.array(invert_perm_o2c_to_c2o(gt), dtype=int)
                match_inv = (pred == gt_as_c2o).sum()
            else:
                match_inv = -1

            match_best = max(match_direct, match_inv)
            ppa_scores.append(match_best / len(gt))
            if match_best == len(gt):
                correct_count += 1

        total = len(df_true)
        acc = (correct_count / total) * 100 if total else 0.0
        mean_ppa = float(np.mean(ppa_scores)) if ppa_scores else 0.0

        print(f"\nTotal images: {total}")
        print(f"Correctly solved: {correct_count}/{total} ({acc:.2f}%)")
        print(f"Average PPA: {mean_ppa:.4f}")

In [3]:
sp_solver = SpectralPuzzleSolver(rows=ROWS, cols=COLS, tau_scale=1.0)


runner = PuzzleRunnerLocal(
    solver=sp_solver,
    pieces_list=all_pieces,
    image_dir=image_path,
    output_dir="output_data",
    output_img_dir="output_data/output_images",
    y_true_csv="input_data/Y_test.csv"
)

out_csv = runner.run_all()
runner.evaluate(out_csv)


Local search for images: 100%|██████████| 100/100 [00:02<00:00, 39.43it/s]

Saved output to output_data\output.csv
Solved images saved to output_data/output_images

Total images: 100
Correctly solved: 0/100 (0.00%)
Average PPA: 0.0993
